<a href="https://colab.research.google.com/github/stgulaa/python/blob/main/%EC%97%B0%EB%A7%90%EC%A0%95%EC%82%B0_%EC%97%91%EC%85%80%EC%97%85%EB%A1%9C%EB%93%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl

In [ ]:
import os
import openpyxl as xls
import pandas as pd
import urllib
import shutil
from posix import remove
import re
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.dimensions import ColumnDimension

In [ ]:
#1
from google.colab import files
url = files.upload()

Saving 2022_근로소득지급명세서_신고파일생성.xlsx to 2022_근로소득지급명세서_신고파일생성.xlsx
Saving 2023홈택스c.xlsx to 2023홈택스c.xlsx


In [ ]:
keyword = "근로소득지급명세서"

files = os.listdir("./")

old_url = [file for file in files if file.endswith(".xlsx") and keyword in file]
new_url = [file for file in files if file.endswith(".xlsx") and keyword not in file]

# print("근로소득지급명세서가 포함된 파일들:", old_url)
# print("그 외 파일들:", new_url)

In [ ]:
def process_excel_file(file_path):
    wb = xls.load_workbook(file_path)

    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # 서식항목 빈 열 지우기 - 매개변수 값넣어서 지우는 경우 하나 끝내고 다시 sheet의 열을 세서 지워짐
        sheet.delete_cols(3)
        sheet.delete_cols(3)
        sheet.delete_cols(3)

        if sheet.max_column >= 5:  # 필요없는 열 전부 삭제
            sheet.delete_cols(5,sheet.max_column+1)

        for row in list(sheet.iter_rows(min_row=2, min_col=1)):
          if not row[0].value:
            sheet.delete_rows(row[0].row)

          elif not row[3].value:
            sheet.delete_rows(row[3].row)

    # F열부터 Q열까지 열 추가
    wb.active.insert_cols(6, amount=11)

    # 변경된 파일 이름으로 저장
    new_filename = os.path.join("./", "2023레코드.xlsx")
    wb.save(new_filename)
    return new_filename

pre_save = process_excel_file("./".join(s for s in new_url))

In [ ]:
#2. 데이터 준비
old_excel = xls.load_workbook('./'.join(s for s in old_url))
new_excel = xls.load_workbook(pre_save)

old_ws = old_excel.active
new_ws = new_excel.active


#2022_근로소득지급명세서 파일에서 비교할 시트가져오기
for ws_names in old_excel.sheetnames:
  if pre_save[2:6] in ws_names:
    compare_sh = ws_names

compare_sh = old_excel[compare_sh]

In [ ]:
#3 갱신된 값이 나오는 시작점 찾기
for i in range(2,new_ws.max_row+1):
  if(new_ws.cell(row=i, column =1).font.color.rgb == 'FFFF0000'): #폰트 빨간색(FFFF0000)
    final_black_cell = i-1
    break

cnt = 2

for row in compare_sh.iter_rows(min_row=3, max_row = final_black_cell+1, min_col=5, max_col=17):
    new_ws.cell(row = cnt, column= 9, value = new_ws.cell(row=cnt, column=2).value)
    new_ws.cell(row = cnt, column= 10, value = row[6].value)
    new_ws.cell(row = cnt, column= 11, value = "=LEFT(C"+str(cnt)+",1)")
    new_ws.cell(row = cnt, column= 12, value = "=MID(C" + str(cnt) + ", FIND(\"(\",C" + str(cnt) + ")+1, LENB(C" + str(cnt) + ")-FIND(\"(\",C" + str(cnt) + ")-1)")
    new_ws.cell(row = cnt, column= 13, value =  "=\"INSERT INTO E21.E2QT_RELAT_FILE_INFO(FILE_FLAG, RECORD_FLAG, SEQ, SYS_FLAG, FILE_COL_NM, FILE_COL, FILE_COL_TYPE, DATA_LENGTH) VALUES ('10','\"&F" + str(cnt) + "&\"','\"&G" + str(cnt) + "&\"','E2','\"&I" + str(cnt) + "&\"','\"&J" + str(cnt) + "&\"','\"&K" + str(cnt) + "&\"','\"&L" + str(cnt) + "&\");\"")
    #new_ws.cell(row = cnt, column= 14, value = row[11].value) #엑셀 파일이 만들어진 후 후처리하는 거라서 없어도 됨
    cnt+=1

In [ ]:
#비교1
def copy_row(new_row, copy_row):
  copy_row = int(re.sub(r"[^0-9]","",copy_row[0].coordinate)) #복사해올 행
  new_ws.cell(row = new_row, column= 9, value = new_ws.cell(row=new_row, column=2).value)
  new_ws.cell(row = new_row, column= 10, value = compare_sh.cell(row=copy_row, column=11).value)
  new_ws.cell(row = new_row, column= 11, value = "=LEFT(C"+str(new_row)+",1)")
  new_ws.cell(row = new_row, column= 12, value = "=MID(C" + str(new_row) + ", FIND(\"(\",C" + str(new_row) + ")+1, LENB(C" + str(new_row) + ")-FIND(\"(\",C" + str(new_row) + ")-1)")
  new_ws.cell(row = new_row, column= 13, value = "=\"INSERT INTO E21.E2QT_RELAT_FILE_INFO(FILE_FLAG, RECORD_FLAG, SEQ, SYS_FLAG, FILE_COL_NM, FILE_COL, FILE_COL_TYPE, DATA_LENGTH) VALUES ('10','\"&F" + str(new_row) + "&\"','\"&G" + str(new_row) + "&\"','E2','\"&I" + str(new_row) + "&\"','\"&J" + str(new_row) + "&\"','\"&K" + str(new_row) + "&\"','\"&L" + str(new_row) + "&\");\"")
  #new_ws.cell(row = new_row, column= 14, value = new_ws.cell(row=new_row, column = 10).value)#엑셀 파일이 만들어진 후 후처리하는 거라서 없어도 됨

name_list = []

for i in range(2,new_ws.max_row+1):
  name_list.append(new_ws.cell(row=i, column=2).value)

for i in range(final_black_cell+1, new_ws.max_row + 1):
    origin_str = str(new_ws.cell(row=i, column=2).value).replace(" ", "")
    cnt = 0
    cnt2 = 0

    for name in range(final_black_cell-1, i-1): # name_list가 0으로 시작하고 2열에 있는 값부터 순서대로 저장하기 때문에 반복문 범위 조정함
      if name_list[name].replace(" ", "") == origin_str:
        cnt += 1

    for row in compare_sh.iter_rows(min_row=final_black_cell+1, max_row = compare_sh.max_row, min_col=2, max_col=2):
      compare_str = str(row[0].value).replace(" ", "")

      matching_characters = sum(1 for a, b in zip(compare_str, origin_str) if a == b)

      if((compare_str[0] == "-" and compare_str[len(compare_str)-1] =="-")):
        if(matching_characters >= 6):
          copy_row(i, row)

      else:
         if(origin_str == compare_str):
            cnt2 +=1
            if cnt == cnt2:
              copy_row(i,row)
              break
         else:
          continue


In [ ]:
def pre_save():
  cnt = 1
  for i in range(2, new_ws.max_row+1):
    new_ws.cell(row = i, column = 5, value = compare_sh['F3'].value)
    new_ws.cell(row = i, column = 6, value = compare_sh['G3'].value)
    new_ws.cell(row = i, column = 7, value = str(cnt).rjust(4,'0'))
    new_ws.cell(row = i, column = 8, value = compare_sh['I3'].value)
    cnt += 1

def auto_format_column(worksheet, columns=None, margin=2): #사용x
    for i, column_cells in enumerate(worksheet.columns):
        is_ok = False
        if columns == None:
            is_ok = True
        elif isinstance(columns, list) and i in columns:
            is_ok = True

        if is_ok:
            length = max(len(str(cell.value)) for cell in column_cells) #byte 수로 계산
            worksheet.column_dimensions[column_cells[0].column_letter].width = length*1.5
    return worksheet

def auto_format_row(worksheet): #자동 행 높이 맞춤
  min = 10000
  for i in range(0, worksheet.max_row+1):
    if worksheet.row_dimensions[i].height is not None and min > worksheet.row_dimensions[i].height:
      min = worksheet.row_dimensions[i].height

  for i in range(0, worksheet.max_row+1):
    worksheet.row_dimensions[i].height = min
  return worksheet

In [ ]:
#최종 저장
pre_save()
auto_format_row(new_ws)

new_excel.save("./final.xlsx")